# **0.3 MTGO Deep Learning**

In [15]:
# Importamos librerías 
import pandas as pd
import numpy as np
import os
import networkx as nx


In [16]:
# Cargar datos 
data_dir = os.path.join("..","data","processed","raw_cleaned")

In [24]:
# Preparar red PPI
ppi = pd.read_csv(os.path.join(data_dir, "ppi_clean.csv"))
G = nx.from_pandas_edgelist(ppi,source='Prot_A', target='Prot_B')
print(f"Nodos: {G.number_of_nodes()}, Aristas: {G.number_of_edges()}\n")


Nodos: 5390, Aristas: 430206



In [25]:
# Cargar matriz GO (función biológica)
GO = pd.read_csv(os.path.join(data_dir, "dataset_completo.csv"), index_col=0) # proteinas como índice
print(f"Dimensiones de la matriz GO: {GO.shape}\n")

Dimensiones de la matriz GO: (5161, 6928)



In [26]:
# Cargar DEG (UP/DOWN)
topo = pd.read_csv(os.path.join(data_dir, "topo_clean.csv"))
deg_dict = topo.set_index("Node_id")["DEG"].to_dict() # diccionario de DEG 'Up' y 'Down'
print(f"Proteínas con anotación DEG: {len(deg_dict)}\n")

Proteínas con anotación DEG: 158



In [47]:
# Crear partición aleatoria
import random
import math
N = len(G.nodes())
H = int(math.sqrt(N)) # Número de iteraciones para la convergencia
#H = 500

nodes = list(G.nodes())
random.shuffle(nodes)
partition = [set(nodes[i::H]) for i in range(H)]
print(f"Se generaron {len(partition)} módulos topológicos.\n")

# aquí cada set es un módulo topológico inicial c_h ∈ C.

Se generaron 73 módulos topológicos.



In [46]:
# Asociar término GO más representativo a cada módulo

def best_go(module_nodes, go_matrix):
    sub = go_matrix.loc[go_matrix.index.intersection(module_nodes)]
    if sub.empty:
        return None
    return sub.sum().idxmax()  # GO más frecuente

module_go = []
for i, mod in enumerate(partition):
    go_term = best_go(mod, GO)
    module_go.append((f"mod_{i}", mod, go_term))
    print(f"{i+1:02d}/{H} → Módulo {i} asignado a GO: {go_term}")

# Esto da una asociacion c_h -> φ_h es el GO más enriquecido en c_h


01/73 → Módulo 0 asignado a GO: GO:0043231_CC
02/73 → Módulo 1 asignado a GO: GO:0043231_CC
03/73 → Módulo 2 asignado a GO: GO:0043231_CC
04/73 → Módulo 3 asignado a GO: GO:0043231_CC
05/73 → Módulo 4 asignado a GO: GO:0043231_CC
06/73 → Módulo 5 asignado a GO: GO:0043231_CC
07/73 → Módulo 6 asignado a GO: GO:0043231_CC
08/73 → Módulo 7 asignado a GO: GO:0043231_CC
09/73 → Módulo 8 asignado a GO: GO:0043231_CC
10/73 → Módulo 9 asignado a GO: GO:0043231_CC
11/73 → Módulo 10 asignado a GO: GO:0043231_CC
12/73 → Módulo 11 asignado a GO: GO:0043231_CC
13/73 → Módulo 12 asignado a GO: GO:0043231_CC
14/73 → Módulo 13 asignado a GO: GO:0043231_CC
15/73 → Módulo 14 asignado a GO: GO:0043231_CC
16/73 → Módulo 15 asignado a GO: GO:0043231_CC
17/73 → Módulo 16 asignado a GO: GO:0043231_CC
18/73 → Módulo 17 asignado a GO: GO:0043231_CC
19/73 → Módulo 18 asignado a GO: GO:0043231_CC
20/73 → Módulo 19 asignado a GO: GO:0043231_CC
21/73 → Módulo 20 asignado a GO: GO:0043231_CC
22/73 → Módulo 21 asign

In [48]:
asignados = [term for _, _, term in module_go if term is not None]
tipos = pd.Series(asignados).str.extract(r'_(\w+)$')[0].value_counts()
print("Distribución de GO asignados por tipo:")
print(tipos)


Distribución de GO asignados por tipo:
0
CC    73
Name: count, dtype: int64


In [49]:
# Contar Up/Down en cada módulo
results = []

for mod_name, mod_nodes, go_term in module_go:
    up = sum(deg_dict.get(p, "") == "Up" for p in mod_nodes)
    down = sum(deg_dict.get(p, "") == "Down" for p in mod_nodes)
    total = len(mod_nodes)
    
    results.append({
        "Modulo": mod_name,
        "GO": go_term,
        "Total proteínas": total,
        "Up": up,
        "Down": down
    })
    print(f"{mod_name}: Total={total}, Up={up}, Down={down}, GO={go_term}")


mod_0: Total=74, Up=2, Down=2, GO=GO:0043231_CC
mod_1: Total=74, Up=1, Down=0, GO=GO:0043231_CC
mod_2: Total=74, Up=2, Down=0, GO=GO:0043231_CC
mod_3: Total=74, Up=0, Down=1, GO=GO:0043231_CC
mod_4: Total=74, Up=1, Down=1, GO=GO:0043231_CC
mod_5: Total=74, Up=1, Down=2, GO=GO:0043231_CC
mod_6: Total=74, Up=1, Down=3, GO=GO:0043231_CC
mod_7: Total=74, Up=1, Down=1, GO=GO:0043231_CC
mod_8: Total=74, Up=2, Down=0, GO=GO:0043231_CC
mod_9: Total=74, Up=0, Down=2, GO=GO:0043231_CC
mod_10: Total=74, Up=1, Down=3, GO=GO:0043231_CC
mod_11: Total=74, Up=3, Down=2, GO=GO:0043231_CC
mod_12: Total=74, Up=0, Down=1, GO=GO:0043231_CC
mod_13: Total=74, Up=1, Down=3, GO=GO:0043231_CC
mod_14: Total=74, Up=0, Down=0, GO=GO:0043231_CC
mod_15: Total=74, Up=0, Down=1, GO=GO:0043231_CC
mod_16: Total=74, Up=0, Down=0, GO=GO:0043231_CC
mod_17: Total=74, Up=0, Down=1, GO=GO:0043231_CC
mod_18: Total=74, Up=2, Down=2, GO=GO:0043231_CC
mod_19: Total=74, Up=3, Down=0, GO=GO:0043231_CC
mod_20: Total=74, Up=1, Down=2

In [50]:
# Convertir a dataframe y mostrar resultados
df_resultados = pd.DataFrame(results)
print("\nResumen final:")
display(df_resultados.head())


Resumen final:


,Modulo,GO,Total proteínas,Up,Down
0,mod_0,GO:0043231_CC,74,2,2
1,mod_1,GO:0043231_CC,74,1,0
2,mod_2,GO:0043231_CC,74,2,0
3,mod_3,GO:0043231_CC,74,0,1
4,mod_4,GO:0043231_CC,74,1,1
